In [1]:
import pandas as pd
import numpy as np
import re

Читаем данные

In [2]:
df = pd.read_csv("./data/hh_full_dataset.csv")

In [3]:
df.head()

,topic_id,topic_creation_date,initial_state,final_state,resume_id,resume_creation_date,profession,gender,year_of_birth,resume_region,...,work_experience_months,resume_skills_list,vacancy_id,vacancy_creation_date,vacancy_region,work_schedule.1,vacancy_employment_type,compensation_from,compensation_to,vacancy_skills_list
0,b41207109c4,2023-08-31 00:00:00.000,response,response,20110t055,2023-05-19 00:00:00.000,"Менеджер по продажам, менеджер по работе с кли...",female,1994,Москва,...,68,['исполнительность'],85t0f82,2023-08-28 00:00:00.000,Москва,remote,full,80000.0,180000.0,"['crm', 'телефонные переговоры', 'ведение пере..."
1,33c19304124c,2023-07-18 00:00:00.000,response,invitation,1186t64t5,2019-06-04 00:00:00.000,"Менеджер по продажам, менеджер по работе с кли...",male,1988,Москва,...,115,"['грамотная устная и письменная речь', 'умение...",82gb451,2023-06-21 00:00:00.000,Москва,full_day,full,60000.0,110000.0,"['доброжелательность', 'телефонные переговоры'..."
2,339614c96c33,2023-08-07 00:00:00.000,response,invitation,1t102tkb4,2022-10-24 00:00:00.000,"Менеджер по продажам, менеджер по работе с кли...",male,1996,Архангельская область,...,52,"['активные продажи', 'навыки продаж', 'ведение...",846bk4k0,2023-08-07 00:00:00.000,Москва,remote,full,45000.0,50000.0,"['телефонные переговоры', 'навыки продаж', 'пр..."
3,3336b00c412,2023-07-07 00:00:00.000,response,response,20b558k11,2023-07-03 00:00:00.000,"Менеджер по продажам, менеджер по работе с кли...",female,1987,Москва,...,141,"['первичная документация', 'бухгалтерская отче...",k80tkbkk,2023-03-15 00:00:00.000,Москва,full_day,full,110000.0,250000.0,[None]
4,b101412c4c96,2023-08-25 00:00:00.000,invitation,invitation,t62tt116,2018-09-14 00:00:00.000,"Менеджер по продажам, менеджер по работе с кли...",female,1998,Ямало-Ненецкий АО,...,42,NaN,85b001b8,2023-08-18 00:00:00.000,Москва,remote,full,0.0,275000.0,"['слепая печать', 'телефонные переговоры', 'на..."


In [4]:
df.columns

Index(['topic_id', 'topic_creation_date', 'initial_state', 'final_state',
       'resume_id', 'resume_creation_date', 'profession', 'gender',
       'year_of_birth', 'resume_region', 'education_level', 'expected_salary',
       'relocation_status', 'business_trip_readiness', 'work_schedule',
       'resume_employment_type', 'work_experience_months',
       'resume_skills_list', 'vacancy_id', 'vacancy_creation_date',
       'vacancy_region', 'work_schedule.1', 'vacancy_employment_type',
       'compensation_from', 'compensation_to', 'vacancy_skills_list'],
      dtype='object')

Удаляем ненужные для создания текста столбцы

In [5]:
columns_to_delete = ["topic_id",
                     "topic_creation_date",
                     "resume_id",
                     "resume_creation_date",
                     "vacancy_id",
                     "vacancy_creation_date"
                     ]

df.drop(axis=1, columns=columns_to_delete, inplace=True)

In [6]:
df.head()

,initial_state,final_state,profession,gender,year_of_birth,resume_region,education_level,expected_salary,relocation_status,business_trip_readiness,work_schedule,resume_employment_type,work_experience_months,resume_skills_list,vacancy_region,work_schedule.1,vacancy_employment_type,compensation_from,compensation_to,vacancy_skills_list
0,response,response,"Менеджер по продажам, менеджер по работе с кли...",female,1994,Москва,higher,65000.0,not_ready,never,full_day,full,68,['исполнительность'],Москва,remote,full,80000.0,180000.0,"['crm', 'телефонные переговоры', 'ведение пере..."
1,response,invitation,"Менеджер по продажам, менеджер по работе с кли...",male,1988,Москва,unfinished_higher,80000.0,not_ready,ready,full_day,full,115,"['грамотная устная и письменная речь', 'умение...",Москва,full_day,full,60000.0,110000.0,"['доброжелательность', 'телефонные переговоры'..."
2,response,invitation,"Менеджер по продажам, менеджер по работе с кли...",male,1996,Архангельская область,unfinished_higher,60000.0,not_ready,never,full_day,full,52,"['активные продажи', 'навыки продаж', 'ведение...",Москва,remote,full,45000.0,50000.0,"['телефонные переговоры', 'навыки продаж', 'пр..."
3,response,response,"Менеджер по продажам, менеджер по работе с кли...",female,1987,Москва,higher,150000.0,not_ready,never,full_day,full,141,"['первичная документация', 'бухгалтерская отче...",Москва,full_day,full,110000.0,250000.0,[None]
4,invitation,invitation,"Менеджер по продажам, менеджер по работе с кли...",female,1998,Ямало-Ненецкий АО,unfinished_higher,70000.0,possible,ready,full_day,full,42,NaN,Москва,remote,full,0.0,275000.0,"['слепая печать', 'телефонные переговоры', 'на..."


Поменяем язык категорий в данных, чтобы удобнее было составлять текст

In [7]:
df["gender"].replace({"female": "женский", 
                      "male": "мужской"}, inplace=True)

df["education_level"].replace({"higher": "высшее", 
                               "unfinished_higher": "неоконченное высшее",
                               "special_secondary": "общее специальное",
                               "secondary": "общее",
                               "bachelor": "бакалавр",
                               "master": "магистр",
                               "candidate": "кандидат наук",
                               "doctor": "доктор наук"}, inplace=True)

df["relocation_status"].replace({"not_ready": "не готов", 
                                 "possible": "готов", 
                                 "desirable": "желательно"}, inplace=True)

df["business_trip_readiness"].replace({"never": "не готов", 
                                       "ready": "готов", 
                                       "sometimes": "готов в некоторых случаях"}, inplace=True)

df["work_schedule"].replace({"full_day": "пятидневная рабочая неделя", 
                             "remote": "удаленная работа", 
                             "flexible": "гибкий график"}, inplace=True)

df["resume_employment_type"].replace({"full": "полная занятость", 
                                      "part": "частичная занятость", 
                                      "project": "проектная занятость", 
                                      "probation": "испытательный срок", 
                                      "volonteer": "волонтер"}, inplace=True)

df["work_schedule.1"].replace({"remote": "удаленная работа", 
                               "full_day": "пятидневная рабочая неделя", 
                               "shift": "работа по сменам", 
                               "flexible": "гибкий график", 
                               "flyInFlyOut": "вахта"}, inplace=True)

df["vacancy_employment_type"].replace({"full": "полная занятость", 
                                      "part": "частичная занятость", 
                                      "project": "проектная занятость", 
                                      "probation": "испытательный срок", 
                                      "volonteer": "волонтер"}, inplace=True)


Преобразуем поля со скиллами

In [8]:
df["vacancy_skills_list"].replace({"[None]":"не указано"}, inplace=True)

In [9]:
df["resume_skills_list"].fillna("не указано", inplace=True)
df["vacancy_skills_list"].fillna("не указано", inplace=True)

In [10]:
df["resume_skills_list"] = df["resume_skills_list"].apply(lambda x: re.sub(r'\'*\[*\]*', '', x))
df["vacancy_skills_list"] = df["vacancy_skills_list"].apply(lambda x: re.sub(r'\'*\[*\]*', '', x))

Заменим пропуски

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 20 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   initial_state            499166 non-null  object 
 1   final_state              499166 non-null  object 
 2   profession               500000 non-null  object 
 3   gender                   500000 non-null  object 
 4   year_of_birth            500000 non-null  int64  
 5   resume_region            500000 non-null  object 
 6   education_level          499898 non-null  object 
 7   expected_salary          489214 non-null  float64
 8   relocation_status        500000 non-null  object 
 9   business_trip_readiness  500000 non-null  object 
 10  work_schedule            493089 non-null  object 
 11  resume_employment_type   500000 non-null  object 
 12  work_experience_months   500000 non-null  int64  
 13  resume_skills_list       500000 non-null  object 
 14  vaca

In [12]:
df["work_schedule"].fillna("не указано", inplace=True)
df["education_level"].fillna("не указано", inplace=True)

Удалим строки в которых неизвестен исход 

In [13]:
df.dropna(subset=["initial_state", "final_state"], inplace=True)

Пропуски в полях с зарплатами

In [14]:
df["expected_salary"].fillna("не указано", inplace=True)
df["compensation_from"].fillna("не указано", inplace=True)
df["compensation_to"].fillna("не указано", inplace=True)

C:\Users\yxtag\AppData\Local\Temp\ipykernel_24848\3505436005.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'не указано' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df["expected_salary"].fillna("не указано", inplace=True)
C:\Users\yxtag\AppData\Local\Temp\ipykernel_24848\3505436005.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'не указано' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df["compensation_from"].fillna("не указано", inplace=True)
C:\Users\yxtag\AppData\Local\Temp\ipykernel_24848\3505436005.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'не указано' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df["compensation_to"

Функция для преобразования строки данных в текст

In [15]:
s = df.iloc[1]

In [16]:
s

initial_state                                                       response
final_state                                                       invitation
profession                 Менеджер по продажам, менеджер по работе с кли...
gender                                                               мужской
year_of_birth                                                           1988
resume_region                                                         Москва
education_level                                          неоконченное высшее
expected_salary                                                      80000.0
relocation_status                                                   не готов
business_trip_readiness                                                готов
work_schedule                                     пятидневная рабочая неделя
resume_employment_type                                      полная занятость
work_experience_months                                                   115

In [17]:
s["compensation_to"]

110000.0

In [18]:
df.columns

Index(['initial_state', 'final_state', 'profession', 'gender', 'year_of_birth',
       'resume_region', 'education_level', 'expected_salary',
       'relocation_status', 'business_trip_readiness', 'work_schedule',
       'resume_employment_type', 'work_experience_months',
       'resume_skills_list', 'vacancy_region', 'work_schedule.1',
       'vacancy_employment_type', 'compensation_from', 'compensation_to',
       'vacancy_skills_list'],
      dtype='object')

In [19]:
# Резюме
cv = ""

cv += "<Имя> <Фамилия>\n\n"

cv += "Пол: " + str(s["gender"]) + "\n"
cv += "Год рождения: " + str(s["year_of_birth"]) + "\n"
cv += str(s["resume_region"]) + "\n\n"

cv += "Желаемая должность: " + str(s["profession"]) + "\n"
cv += "Желаемая зарплата: " + str(s["expected_salary"]) + " рублей\n"
cv += "Возможность релокации: " + str(s["relocation_status"]) + "\n"
cv += "Возможность командировок: " + str(s["business_trip_readiness"]) + "\n"
cv += "Рабочий график: " + str(s["work_schedule"]) + "\n"
cv += "Тип трудоустройства: " + str(s["resume_employment_type"]) + "\n"

cv += "Образование: " + str(s["education_level"]) + "\n"
cv += "Навыки: " + s["resume_skills_list"] + "\n"


cv += "Опыт работы: " + str(s["work_experience_months"] // 12) + " лет и " + str(s["work_experience_months"] % 12) + " месяцев\n"

# Вакансия
vaccancy = ""

vaccancy += "Позиция: " + str(s["profession"]) + "\n"
vaccancy += "Регион: " + str(s["vacancy_region"]) + "\n"
vaccancy += "Рабочий график: " + str(s["work_schedule.1"]) + "\n"
vaccancy += "Тип трудоустройства: " + str(s["vacancy_employment_type"]) + "\n"
vaccancy += "Зарплата от: " + str(s["compensation_from"]) + " рублей\n"
vaccancy += "Зарплата до: " + str(s["compensation_to"]) + " рублей\n"
vaccancy += "Требуемые навыки: " + s["vacancy_skills_list"]


In [20]:
print(vaccancy)

Позиция: Менеджер по продажам, менеджер по работе с клиентами
Регион: Москва
Рабочий график: пятидневная рабочая неделя
Тип трудоустройства: полная занятость
Зарплата от: 60000.0 рублей
Зарплата до: 110000.0 рублей
Требуемые навыки: доброжелательность, телефонные переговоры, пользователь пк, умение работать в команде, внимательность, грамотная речь


In [21]:
print(cv)

<Имя> <Фамилия>

Пол: мужской
Год рождения: 1988
Москва

Желаемая должность: Менеджер по продажам, менеджер по работе с клиентами
Желаемая зарплата: 80000.0 рублей
Возможность релокации: не готов
Возможность командировок: готов
Рабочий график: пятидневная рабочая неделя
Тип трудоустройства: полная занятость
Образование: неоконченное высшее
Навыки: грамотная устная и письменная речь, умение вести переговоры, умение работать в команде, ведение деловой переписки, работа с первичной документацией(сф, тн, ттн, договоры), умение работать с большим объёмом информации, опытный пользователь пк, знание ms office
Опыт работы: 9 лет и 7 месяцев



In [22]:
def create_cv_text(s):

    cv = ""

    cv += "<Имя> <Фамилия>\n\n"

    cv += "Пол: " + str(s["gender"]) + "\n"
    cv += "Год рождения: " + str(s["year_of_birth"]) + "\n"
    cv += str(s["resume_region"]) + "\n\n"

    cv += "Желаемая должность: " + str(s["profession"]) + "\n"
    cv += "Желаемая зарплата: " + str(s["expected_salary"]) + " рублей\n"
    cv += "Возможность релокации: " + str(s["relocation_status"]) + "\n"
    cv += "Возможность командировок: " + str(s["business_trip_readiness"]) + "\n"
    cv += "Рабочий график: " + str(s["work_schedule"]) + "\n"
    cv += "Тип трудоустройства: " + str(s["resume_employment_type"]) + "\n"

    cv += "Образование: " + str(s["education_level"]) + "\n"
    cv += "Навыки: " + s["resume_skills_list"] + "\n"


    cv += "Опыт работы: " + str(s["work_experience_months"] // 12) + " лет и " + str(s["work_experience_months"] % 12) + " месяцев\n"

    return cv


def create_vaccancy_text(s):

    vaccancy = ""

    vaccancy += "Позиция: " + str(s["profession"]) + "\n"
    vaccancy += "Регион: " + str(s["vacancy_region"]) + "\n"
    vaccancy += "Рабочий график: " + str(s["work_schedule.1"]) + "\n"
    vaccancy += "Тип трудоустройства: " + str(s["vacancy_employment_type"]) + "\n"
    vaccancy += "Зарплата от: " + str(s["compensation_from"]) + " рублей\n"
    vaccancy += "Зарплата до: " + str(s["compensation_to"]) + " рублей\n"
    vaccancy += "Требуемые навыки: " + s["vacancy_skills_list"]

    return vaccancy

In [34]:
def create_cv_text_2(s):

    cv = ""

    cv += "<Имя> <Фамилия>\n\n"

    cv += "Пол: " + str(s["gender"]) + "\n"
    cv += "Год рождения " + str(s["year_of_birth"]) + "\n"
    cv += "Проживаю в " + str(s["resume_region"]) + "\n\n"

    cv += "Хочу устроиться на позицию " + str(s["profession"]) + ". "
    cv += "Исходя из моих профессиональных навыков и опыта я рассчитваю на зарплату в размере " + str(s["expected_salary"]) + " рублей. "
    cv += "К релокации " + str(s["relocation_status"]) + ". "
    cv += "К командировкам " + str(s["business_trip_readiness"]) + ". "
    cv += "Желамый рабочий график - " + str(s["work_schedule"]) + ". "
    cv += "Желаемый тип трудоустройства - " + str(s["resume_employment_type"]) + "\n"

    cv += "Имею " + str(s["education_level"]) + " образование. "
    cv += "Обладаю следующими навыками: " + s["resume_skills_list"] + ".\n"


    cv += "Мой общий трудовой стаж составляет " + str(s["work_experience_months"] // 12) + " лет и " + str(s["work_experience_months"] % 12) + " месяцев\n"

    return cv


def create_vaccancy_text_2(s):

    vaccancy = ""

    vaccancy += "Ищем специалиста на позицию " + str(s["profession"]) + ". "
    vaccancy += "В " + str(s["vacancy_region"]) + ". "
    vaccancy += "Условия работы: " + str(s["work_schedule.1"]) + " и "
    vaccancy += str(s["vacancy_employment_type"]) + ". "
    vaccancy += "Предлагаем зарплату от: " + str(s["compensation_from"])
    vaccancy += " до: " + str(s["compensation_to"]) + " рублей. "
    vaccancy += "От соискателя на данную позицию требуются следующие навыки: " + s["vacancy_skills_list"] + "."

    return vaccancy

In [25]:
df

,initial_state,final_state,profession,gender,year_of_birth,resume_region,education_level,expected_salary,relocation_status,business_trip_readiness,work_schedule,resume_employment_type,work_experience_months,resume_skills_list,vacancy_region,work_schedule.1,vacancy_employment_type,compensation_from,compensation_to,vacancy_skills_list
0,response,response,"Менеджер по продажам, менеджер по работе с кли...",женский,1994,Москва,высшее,65000.0,не готов,не готов,пятидневная рабочая неделя,полная занятость,68,исполнительность,Москва,удаленная работа,полная занятость,80000.0,180000.0,"crm, телефонные переговоры, ведение переговоро..."
1,response,invitation,"Менеджер по продажам, менеджер по работе с кли...",мужской,1988,Москва,неоконченное высшее,80000.0,не готов,готов,пятидневная рабочая неделя,полная занятость,115,"грамотная устная и письменная речь, умение вес...",Москва,пятидневная рабочая неделя,полная занятость,60000.0,110000.0,"доброжелательность, телефонные переговоры, пол..."
2,response,invitation,"Менеджер по продажам, менеджер по работе с кли...",мужской,1996,Архангельская область,неоконченное высшее,60000.0,не готов,не готов,пятидневная рабочая неделя,полная занятость,52,"активные продажи, навыки продаж, ведение перег...",Москва,удаленная работа,полная занятость,45000.0,50000.0,"телефонные переговоры, навыки продаж, прямые п..."
3,response,response,"Менеджер по продажам, менеджер по работе с кли...",женский,1987,Москва,высшее,150000.0,не готов,не готов,пятидневная рабочая неделя,полная занятость,141,"первичная документация, бухгалтерская отчетнос...",Москва,пятидневная рабочая неделя,полная занятость,110000.0,250000.0,не указано
4,invitation,invitation,"Менеджер по продажам, менеджер по работе с кли...",женский,1998,Ямало-Ненецкий АО,неоконченное высшее,70000.0,готов,готов,пятидневная рабочая неделя,полная занятость,42,не указано,Москва,удаленная работа,полная занятость,0.0,275000.0,"слепая печать, телефонные переговоры, навыки п..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,response,response,Менеджер по закупкам,мужской,1991,Москва,высшее,120000.0,не готов,готов в некоторых случаях,пятидневная рабочая неделя,полная занятость,133,"ответственность, 1с: предприятие 8, гоз, оформ...",Москва,пятидневная рабочая неделя,полная занятость,130000.0,150000.0,"высокая энергичность и инициативность, чтение ..."
499996,response,response,Менеджер по закупкам,мужской,1984,Москва,высшее,80000.0,не готов,готов,пятидневная рабочая неделя,полная занятость,90,работоспособность,Москва,пятидневная рабочая неделя,полная занятость,90000.0,90000.0,опыт работы в автоматизированных корпоративных...
499997,response,response,Менеджер по закупкам,женский,1986,Нижегородская область,неоконченное высшее,50000.0,готов,готов,пятидневная рабочая неделя,полная занятость,179,"microsoft office, photoshop, correl drow, декл...",Москва,пятидневная рабочая неделя,полная занятость,93000.0,145000.0,"мотивация персонала, управление персоналом, по..."
499998,response,discard,Менеджер по закупкам,мужской,1977,Саратовская область,высшее,20000.0,не готов,готов,пятидневная рабочая неделя,полная занятость,0,обучаемость,Москва,удаленная работа,полная занятость,60000.0,не указано,"логистика поставок, маркетплейсы, производство..."


In [35]:
new_df = df.copy()

new_df["cv_text"] = new_df.apply(lambda x: create_cv_text_2(x), axis=1)
new_df["vaccancy_text"] = new_df.apply(lambda x: create_vaccancy_text_2(x), axis=1)

new_df = new_df[["cv_text", "vaccancy_text", "initial_state", "final_state"]]

In [36]:
new_df

,cv_text,vaccancy_text,initial_state,final_state
0,<Имя> <Фамилия>\n\nПол: женский\nГод рождения ...,Ищем специалиста на позицию Менеджер по продаж...,response,response
1,<Имя> <Фамилия>\n\nПол: мужской\nГод рождения ...,Ищем специалиста на позицию Менеджер по продаж...,response,invitation
2,<Имя> <Фамилия>\n\nПол: мужской\nГод рождения ...,Ищем специалиста на позицию Менеджер по продаж...,response,invitation
3,<Имя> <Фамилия>\n\nПол: женский\nГод рождения ...,Ищем специалиста на позицию Менеджер по продаж...,response,response
4,<Имя> <Фамилия>\n\nПол: женский\nГод рождения ...,Ищем специалиста на позицию Менеджер по продаж...,invitation,invitation
...,...,...,...,...
499995,<Имя> <Фамилия>\n\nПол: мужской\nГод рождения ...,Ищем специалиста на позицию Менеджер по закупк...,response,response
499996,<Имя> <Фамилия>\n\nПол: мужской\nГод рождения ...,Ищем специалиста на позицию Менеджер по закупк...,response,response
499997,<Имя> <Фамилия>\n\nПол: женский\nГод рождения ...,Ищем специалиста на позицию Менеджер по закупк...,response,response
499998,<Имя> <Фамилия>\n\nПол: мужской\nГод рождения ...,Ищем специалиста на позицию Менеджер по закупк...,response,discard


In [37]:
print(new_df.iloc[23]["cv_text"])

<Имя> <Фамилия>

Пол: женский
Год рождения 1977
Проживаю в Московская область

Хочу устроиться на позицию Менеджер по продажам, менеджер по работе с клиентами. Исходя из моих профессиональных навыков и опыта я рассчитваю на зарплату в размере 82000.0 рублей. К релокации не готов. К командировкам не готов. Желамый рабочий график - пятидневная рабочая неделя. Желаемый тип трудоустройства - полная занятость
Имею общее специальное образование. Обладаю следующими навыками: работа с возражениями, проведение презентаций, битрикс 24, навыки продаж, поиск и привлечение клиентов, microsoft office, деловая коммуникация, грамотная речь, развитие продаж, в2с продажи, 1с: предприятие 8, 1с: торговля, консультирование клиентов по телефону, amocrm, допродажи, активные продажи, наработка и развитие клиентской базы, продажи по телефону, привлечение клиентов, проведение презентаций удаленно, excel, продажи дополнительных услуг, 1с: торговля и склад, в2в продажи, деловое общение, eme, ведение клиентской б

In [193]:
new_df.to_csv("./data/final_dataset_2.csv")